In [25]:
import uproot
import awkward as ak
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Open the ROOT files
events = uproot.open("/fast_scratch_1/jbohm/cell_particle_deposit_learning/delta/delta_root_files/delta_full.root" + ":EventTree")
cell_geo_tree = uproot.open("/data/atlas/data/rho_delta/rho_small.root" + ":CellGeo")

# Find the first event with 8 or more truth particles
min_parts = 8
n_truth_part = events["nTruthPart"].array()
first_index = ak.firsts(ak.where(n_truth_part >= min_parts))
if not ak.is_valid(first_index):
    raise ValueError(f"No event with {min_parts} or more truth particles was found.")
print(f"Event {first_index} has {n_truth_part[first_index]} truth particles.")

# Load data for the specific event
event_data = events.arrays(entry_start=first_index[0], entry_stop=first_index[0] + 1)
cell_geo_data = cell_geo_tree.arrays()

# Convert eta, phi to cartesian coordinates
def eta_phi_to_xyz(eta, phi, rPerp):
    theta = 2 * np.arctan(np.exp(-eta))
    return rPerp * np.cos(phi), rPerp * np.sin(phi), rPerp / np.tan(theta)

# Extract cell IDs and geometric data
cell_ids = cell_geo_data["cell_geo_ID"][0]
eta, phi, rPerp = cell_geo_data["cell_geo_eta"][0], cell_geo_data["cell_geo_phi"][0], cell_geo_data["cell_geo_rPerp"][0]

# Convert eta, phi to cartesian coordinates
x, y, z = eta_phi_to_xyz(eta, phi, rPerp)

# Create a dictionary to map cell IDs to their x, y, z coordinates
cell_id_to_xyz = {cell_id: (x_val, y_val, z_val) for cell_id, x_val, y_val, z_val in zip(cell_ids, x, y, z)}

# Process Energy Deposits
# Load energy deposits and cell IDs from the event
hits_truth_id = event_data["cluster_cell_ID"][0]  # Update this if the structure is different
hits_truth_e = event_data["cluster_cell_E"][0] * 1000  # Update and adjust units if necessary
print(f"hits_truth_id: {hits_truth_id}")
print(f"hits_truth_e: {hits_truth_e}")

# Ensure that we're working with flat arrays for zip operation
hits_truth_id_flat = ak.flatten(hits_truth_id)
hits_truth_e_flat = ak.flatten(hits_truth_e)

# Filter cells with no energy deposited and create a dictionary mapping cell IDs to energy deposits
cell_id_to_energy = {cell_id: energy for cell_id, energy in zip(hits_truth_id_flat, hits_truth_e_flat) if energy > 0}
print(f"cell_id_to_energy: {cell_id_to_energy}")

# Determine Color Based on Energy Deposit and Particle Type
# Define color maps for energy deposit and particle types
energy_color_map = plt.get_cmap('viridis')  # Change as needed
particle_color_map = {5: 'yellow', 6: 'blue', '5_and_6': 'green', 'other': 'red'}

# Load particle types (Update this if the structure is different)
if "cluster_cell_hitsTruthIndex" in event_data.fields:
    hits_truth_index = event_data["cluster_cell_hitsTruthIndex"][0][0]
else:
    hits_truth_index = ak.Array([])  # Fallback to an empty array if data is not present

# Assign colors based on energy deposit and particle type
cell_id_to_color_energy = {cell_id: energy_color_map(energy / max(cell_id_to_energy.values())) for cell_id, energy in cell_id_to_energy.items()}
default_color = 'gray'  # Default color for undefined particle types
cell_id_to_color_particle = {
    cell_id: particle_color_map.get(particle_type[0], default_color) if len(particle_type) == 1 else particle_color_map.get('5_and_6', default_color)
    for cell_id, particle_type in zip(hits_truth_id_flat, hits_truth_index)
    if cell_id in cell_id_to_energy  # Ensure the cell has an energy deposit
}

# Prepare the data for plotting
# Filtering the coordinates and assigning colors for cells with energy deposits
xyz = np.array([cell_id_to_xyz[cell_id] for cell_id in cell_id_to_energy.keys()])
colors_energy = np.array([cell_id_to_color_energy[cell_id] for cell_id in cell_id_to_energy.keys()])
colors_particle = np.array([cell_id_to_color_particle.get(cell_id, default_color) for cell_id in cell_id_to_energy.keys()])

# Print final results for inspection
print(f"xyz: {xyz}")
print(f"colors_energy: {colors_energy}")
print(f"colors_particle: {colors_particle}")

Event [161730] has [8] truth particles.
hits_truth_id: [[757123080, 757123078, 757123082, ..., 757126662, 757126666, 757127174]]
hits_truth_e: [[465, 45, 72, 177, 213, 18, 5.03, ..., 65.2, 48.9, 30, 26, 9.03, 22, 20.9]]
cell_id_to_energy: {757123080: 465.21747, 757123078: 45.012135, 757123082: 71.98224, 757122568: 177.2097, 757123592: 212.52643, 757122566: 18.022118, 757123590: 5.030082, 757122570: 16.932009, 757123594: 71.98224, 754980872: 103.19368, 759175200: 358.05237, 759175202: 54.098866, 759175204: 6.981793, 759175206: 450.789, 757124104: 37.00452, 759175208: 291.8611, 759174694: 82.75066, 759174692: 48.155483, 759174696: 122.71199, 757121032: 13.927127, 757121544: 60.92904, 757122056: 122.04689, 757125126: 108.01599, 757125128: 391.70834, 757125130: 37.910717, 757124616: 126.07378, 757125640: 264.28473, 757124618: 8.014569, 757125642: 13.010838, 754981384: 93.98394, 759175712: 206.8128, 759175714: 20.949883, 759175716: 6.041785, 759175718: 28.97759, 757126152: 77.241394, 757126

In [ ]:
# Plot Making

# Define the range for x, y, z axis
x_limits = (-3000, 3000)
y_limits = (-3000, 3000)
z_limits = (-6000, 6000)

# 1. Plot the Current Point Cloud
fig = plt.figure(figsize=(15, 5))  # Adjust the figure size as needed
ax1 = fig.add_subplot(131, projection='3d')
ax1.scatter(x, y, z, color='b')
ax1.set_title(f'Current Point Cloud\nTotal Points: {len(x)}')
ax1.set_xlim(x_limits)
ax1.set_ylim(y_limits)
ax1.set_zlim(z_limits)
ax1.set_box_aspect([1,1,2])  # Aspect ratio is 1:1:2
ax1.set_xlabel('X Label')
ax1.set_ylabel('Y Label')
ax1.set_zlabel('Z Label')
ax1.view_init(elev=45, azim=15, roll=90)

# 2. Plot Point Cloud Colored by Total Adjusted Energy Deposit
ax2 = fig.add_subplot(132, projection='3d')
scatter = ax2.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c=colors_energy)
ax2.set_title(f'Colored by Energy Deposit\nTotal Points: {len(xyz)}')
ax2.set_xlim(x_limits)
ax2.set_ylim(y_limits)
ax2.set_zlim(z_limits)
ax2.set_box_aspect([1,1,2])
ax2.set_xlabel('X Label')
ax2.set_ylabel('Y Label')
ax2.set_zlabel('Z Label')
ax2.view_init(elev=45, azim=15, roll=90)
cbar = plt.colorbar(scatter, ax=ax2, shrink=0.5, aspect=10)
cbar.set_label('Energy Deposit')

# 3. Plot Point Cloud Colored by the Particle Type
ax3 = fig.add_subplot(133, projection='3d')
scatter = ax3.scatter(xyz[:, 0], xyz[:, 1], xyz[:, 2], c=colors_particle)
ax3.set_title(f'Colored by Particle Type\nTotal Points: {len(xyz)}')
ax3.set_xlim(x_limits)
ax3.set_ylim(y_limits)
ax3.set_zlim(z_limits)
ax3.set_box_aspect([1,1,2])
ax3.set_xlabel('X Label')
ax3.set_ylabel('Y Label')
ax3.set_zlabel('Z Label')
ax3.view_init(elev=45, azim=15, roll=90)
cbar = plt.colorbar(scatter, ax=ax3, shrink=0.5, aspect=10)
cbar.set_label('Particle Type')

# Adjust layout
plt.tight_layout()

# Show the plots
plt.show()
